In [1]:
import json
import numpy as np

def power_method(D):
    x_prev = np.random.uniform(size=D.shape[0])
    x_prev /= np.linalg.norm(x_prev)
    
    while True:
        x_cur = D.dot(x_prev)
        lam_cur = np.linalg.norm(x_cur)
        x_cur /= np.linalg.norm(x_cur)

        if np.linalg.norm(x_cur - x_prev) < 1e-5:
            return x_cur / x_cur.sum(), lam_cur

        x_prev = x_cur

def compute_CR(lam_max, N):
    CI = (lam_max - N) / (N-1)
    MRCI = [0, 0, 0.52, 0.89, 1.11, 1.25, 1.35, 1.4, 1.45, 1.49, 1.52, 1.54, 1.56, 1.58, 1.59]
    THRESH = [0, 0, 0.05, 0.08, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
    CR = CI / MRCI[N-1]
    thresh = THRESH[N-1]
    if CR > thresh:
        return f"CR(={CR:.3f}) > {thresh} - МПП неузгоджена"
    elif CR > 0:
        return f"CR(={CR:.3f}) <= {thresh} - МПП допустимо узгоджена"
    else:
        return f"CR = 0.0 - МПП узгоджена"

def distributive_method(W_local, w_crits):
    A = W_local
    wC = w_crits
    R = A / A.sum(0)
    w_glob = (wC * R).sum(1)
    return w_glob

In [2]:
def solve(filename):
    try:
        with open(filename, "r") as fp:
            problem_json = json.load(fp)
    except Exception as exc:
        print(exc)

    p = problem_json["p"]
    m = problem_json["m"]
    w_crits = np.array([problem_json["wCrits"]])
    mpps = problem_json["MPPs"]

    strings = []
    for i_hier in range(p-1):
        strings += [f"\n-- Hierarchy level {i_hier + 2} --"]
        N, N = np.array(mpps[i_hier][0]).shape
        M = len(mpps[i_hier])
        W_local = np.empty((M, N))
        for i_mpp, mpp in enumerate(mpps[i_hier]):
            mpp = np.array(mpp)
            strings += [str(mpp)]
            w_local, lam_max = power_method(mpp)
            strings += [compute_CR(lam_max, w_local.shape[0])]
            W_local[i_mpp] = w_local
        
        w_global = distributive_method(W_local, w_crits)
        w_crits = np.array([w_global])
        strings += [f"Global weights: {w_global}"]
    print("\n".join(strings))

In [3]:
solve('./data/lab_1_sample_input.json')


-- Hierarchy level 2 --
[[1.    0.333 0.333 0.333]
 [3.    1.    1.    1.   ]
 [3.    1.    1.    0.666]
 [3.    1.    1.5   1.   ]]
CR(=0.007) <= 0.08 - МПП допустимо узгоджена
[[1.    0.333 0.666 0.333]
 [3.    1.    2.    1.   ]
 [1.5   0.5   1.    0.666]
 [3.    1.    1.5   1.   ]]
CR(=0.003) <= 0.08 - МПП допустимо узгоджена
[[1.    3.    1.    0.333]
 [0.333 1.    0.333 0.111]
 [1.    3.    1.    0.111]
 [3.    9.    9.    1.   ]]
CR(=0.057) <= 0.08 - МПП допустимо узгоджена
Global weights: [0.34861283 0.3487798  0.30260737]

-- Hierarchy level 3 --
[[1.    3.    5.   ]
 [0.333 1.    2.   ]
 [0.2   0.5   1.   ]]
CR(=0.003) <= 0.05 - МПП допустимо узгоджена
[[1.   4.   2.  ]
 [0.25 1.   0.5 ]
 [0.5  2.   1.  ]]
CR(=0.000) <= 0.05 - МПП допустимо узгоджена
[[1.    3.    5.   ]
 [0.333 1.    2.5  ]
 [0.2   0.4   1.   ]]
CR(=0.017) <= 0.05 - МПП допустимо узгоджена
Global weights: [0.32204152 0.35390303 0.32405545]
